In [80]:
import pandas as pd
import sys
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.neural_network import MLPRegressor

sys.path.append("../")
import src.data_utils as d_u
import src.feats_generation as f_g

В файле baseline.ipynb было показано, что использование эмбеддинга для сообщения коммита позволяет увеличить точность моделей. Попробуем также tfidf с уменьшением размерности, а также предобученный мультиязыковой кодировщик предложений на основе Transformer и CNN

Загрузим датасет и предобработаем категориальные фичи

In [81]:
df = pd.read_csv("../data/raw/АВСОФТ_тест_ML_приложение.csv")
df.drop(columns=["commit_hash"], inplace=True)
df = pd.concat([df, pd.get_dummies(df.repository_name)], axis=1)
df.drop(columns=["repository_name"], inplace=True)
df = f_g.ohe(df, "commit_author")
df["commit_date"] = pd.to_datetime(df.commit_date)

df = f_g.encode_work_days(df)
df = f_g.encode_work_hours(df)

In [82]:
if "commit_date" in df.columns:
    df.drop(columns=["commit_date"], inplace=True)

In [83]:
X = df.drop(columns=["commit_message", "bugs"])
y = df.bugs

Сначала сразу посмотрим результат модели с эмбеддингами сообщений, которые мы получаем с помощью предобученной мультиязыковой модели

In [84]:
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')

In [85]:
msg_embs = model.encode(df.commit_message.values)

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_idxs, test_idxs = X_train.index, X_test.index

In [87]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [88]:
X_train = np.concatenate((X_train, msg_embs[train_idxs]), axis=1)
X_test = np.concatenate((X_test, msg_embs[test_idxs]), axis=1)

In [89]:
zoo_models = [
    LinearRegression(),
    SGDRegressor(random_state=42),
    Ridge(random_state=42),
    Lasso(random_state=42),
    RandomForestRegressor(random_state=42),
    GradientBoostingRegressor(random_state=42),
    SVR(),
    MLPRegressor(random_state=42, max_iter=100000)
    ]

for model in zoo_models:
    print(f"Модель - {model}")
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    print(f"Среднее квадратное отклонение: {mean_squared_error(y_test, preds)}")
    # print(f"Среднее абсолютное отклонение: {mean_absolute_error(y_test, preds)}")
    print()

Модель - LinearRegression()
Среднее квадратное отклонение: 3.9255156937438884

Модель - SGDRegressor(random_state=42)
Среднее квадратное отклонение: 1.5442650283714352

Модель - Ridge(random_state=42)
Среднее квадратное отклонение: 1.3328333886580885

Модель - Lasso(random_state=42)
Среднее квадратное отклонение: 2.5254327662437523

Модель - RandomForestRegressor(random_state=42)
Среднее квадратное отклонение: 0.86288125

Модель - GradientBoostingRegressor(random_state=42)
Среднее квадратное отклонение: 0.7879865467367375

Модель - SVR()
Среднее квадратное отклонение: 2.155657021426432

Модель - MLPRegressor(max_iter=100000, random_state=42)
Среднее квадратное отклонение: 1.7726122043258286



Посмотрим среднее квадратичное отклонение на кросс-валидации

In [48]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
ss = StandardScaler()
X_eval = ss.fit_transform(X)
X_eval = np.concatenate((X_eval, msg_embs), axis=1)

for model in zoo_models:
    print(f"Модель - {model}")
    mse_mean = -np.mean(cross_val_score(model, X_eval, y, cv=cv, scoring="neg_mean_squared_error"))
    print(f"Среднее квадратное отклонение: {mse_mean}")
    # preds = model.predict(X_test)
    # print(f"Среднее квадратное отклонение: {mean_squared_error(y_test, preds)}")
    # print(f"Среднее абсолютное отклонение: {mean_absolute_error(y_test, preds)}")
    print()

Модель - LinearRegression()
Среднее квадратное отклонение: 3.6631705664438847

Модель - SGDRegressor(random_state=42)
Среднее квадратное отклонение: 1.5791955326983218

Модель - Ridge(random_state=42)
Среднее квадратное отклонение: 1.3792463938249164

Модель - Lasso(random_state=42)
Среднее квадратное отклонение: 3.4774931163795175

Модель - RandomForestRegressor(random_state=42)
Среднее квадратное отклонение: 1.7560374454365077

Модель - GradientBoostingRegressor(random_state=42)
Среднее квадратное отклонение: 1.470161025026534

Модель - SVR()
Среднее квадратное отклонение: 2.7170945885316122



In [54]:
dim_reduction = [TruncatedSVD, PCA]
components = [5, 10, 20, 50, 100, 300]
dim_red_comps = [(i, j) for i in dim_reduction for j in components]

In [58]:
for model in zoo_models:
    print(f"Модель - {model}")
    for red, comp in dim_red_comps:
        print(f"{red}________{comp}")
        ss = StandardScaler()
        X_eval = ss.fit_transform(X)
        red_embs = red(n_components=comp).fit_transform(msg_embs)
        X_eval = np.concatenate((X_eval, red_embs), axis=1)
        mse_mean = -np.mean(cross_val_score(model, X_eval, y, cv=cv, scoring="neg_mean_squared_error"))
        print(f"Среднее квадратное отклонение: {mse_mean}")
    print("__________________________________________________")

Модель - LinearRegression()
<class 'sklearn.decomposition._truncated_svd.TruncatedSVD'>________5
Среднее квадратное отклонение: 1.6746896843097763
<class 'sklearn.decomposition._truncated_svd.TruncatedSVD'>________10
Среднее квадратное отклонение: 1.4399618746752116
<class 'sklearn.decomposition._truncated_svd.TruncatedSVD'>________20
Среднее квадратное отклонение: 1.4717938529815442
<class 'sklearn.decomposition._truncated_svd.TruncatedSVD'>________50
Среднее квадратное отклонение: 1.5749672802220611
<class 'sklearn.decomposition._truncated_svd.TruncatedSVD'>________100
Среднее квадратное отклонение: 1.983164210492
<class 'sklearn.decomposition._truncated_svd.TruncatedSVD'>________300
Среднее квадратное отклонение: 3.921372149822684
<class 'sklearn.decomposition._pca.PCA'>________5
Среднее квадратное отклонение: 1.6157300136772275
<class 'sklearn.decomposition._pca.PCA'>________10
Среднее квадратное отклонение: 1.4822063023231031
<class 'sklearn.decomposition._pca.PCA'>________20
Сред

В данный момент можно сделать вывод, что лучший результат достигается при снижении размерности эмбеддинга получаемого предварительно обученной моделью с помощью TruncatedSVD

Попробуем подобрать наиболее подходящую итоговую размерность эмбеддинга

In [62]:
X

,agent,conductor,dockers,mlm,sensor,standard,Alice,Bob,Carol,Dabe,Eve,Mallory,Peggy,Trudy,Victor,Wendy,no_work_d,work_d,no_work_h,work_h
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1
315,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1
316,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1
317,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1


In [95]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
model = MLPRegressor(hidden_layer_sizes=(300, 150, 25,), random_state=42, max_iter=100000)

for n in [5, 10, 15, 20, 25, 30, 40, 50, 100, 200, 250, -1]:
    splits = cv.split(X)
    tmp_lst = []
    for i_train, i_test in splits:
        if n != -1:
            svd = TruncatedSVD(n_components=n)
            msg_embs_train = svd.fit_transform(msg_embs[i_train])
            msg_embs_test = svd.transform(msg_embs[i_test])
        else:
            msg_embs_train = msg_embs[i_train]
            msg_embs_test = msg_embs[i_test]
        X_train, X_test, y_train, y_test = X.loc[i_train], X.loc[i_test], y[i_train], y[i_test]
        # ss = StandardScaler()
        # X_train = ss.fit_transform(X_train)
        # X_test = ss.transform(X_test)
        X_train_emb = np.concatenate((X_train, msg_embs_train), axis=1)
        X_test_emb = np.concatenate((X_test, msg_embs_test), axis=1)
        model.fit(X_train_emb, y_train)
        preds = model.predict(X_test_emb)
        tmp_lst.append(mean_squared_error(y_test, preds))
    print(f"{n} - {np.mean(tmp_lst)}")
    

5 - 1.818188605536184
10 - 1.3885376721725884
15 - 1.243824744753063
20 - 1.1993175413434083
25 - 1.2646591428309857
30 - 1.2581325392105236
40 - 1.209634622731476
50 - 1.1949957423631052
100 - 1.2411780621861077
200 - 1.3845428980240941
250 - 1.4015651592658023
-1 - 1.218504096837509


Ячейки выше показывает, что при понижении размерности можно достичь примерн такого же качества, что и при эмбеддингеоригинального размера, но проще ничего не трогать и получать аналогичное качество работы модели или даже выше. Также StandardScaler так же можно не применять, при его использовании среднее квадратичное отклонение на кросс-валидации слегка увеличивается, как на бустинге, так и на линейной модели с регуляризацией.

Лучше всего себя показала нейронная сеть с понижением размера эмбеддинга. Также можно не понижать размерность эмбеддинга сообщения коммита, но в таком случае необходимо увеличивать количество слоев. Следующими по качеству предсказания идут Ridge и GradientBoostingRegressor.

Отдельно проверять tfidf в данный момент уже не буду, так как эмбеддинги предложений получаемые с помощью предобученной модели сразу показали гораздо лучший результат.